# Assignment 2 Notebook Overview

> Note 1: for answers with Python, display both codes and results clearly. You may explain 
answers with either comments, markdowns or print() function.

> Note 2: for answers with manual calculation, please display all calculation steps clearly. 

> Note 3: round all numerical answers to 2 decimal places.    

## 0. Environment Setup and First Look

Load helper libraries, style the plotting backend, and pull the shared `smoking.csv` dataset for reuse across later sections.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score  

### First Look